# Newton Optimization


This notebook has an example of Neton Optimization tecnique on a Multilayer Perceptron using NumPy.


but first wee need to undestand what is a Newton Optimization Technique and how is it dfferent from the Gradient Descent Tecnique gernally used for optimizing a Neural Network.

In [1]:
import numpy as np
np.seterr('ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

# Single Perceptron

In [2]:
# Here I am building a normal forward propogation for a single perceptron
# and the various function i would use.

In [3]:
def sigmoid(z):
    #this is the non-linear activation function
    return (1 / (1 + np.exp(-z)))

In [4]:
#############################################
## Declaring Traing Data        ############
#############################################
X_train = np.array([[0,0],[1,0],[0,1],[1,1]])
Y_train = np.array([[1],[0],[0],[1]])



In [5]:
class Loss:
    '''
    Class containing loss functions
    '''
    
    def __init__(self):
        return
    
    def sq_loss(self , pred ,Y):
        loss = Y - pred
        loss_dash = -1 * loss
        loss = 1/2 * np.sum(loss * loss)
        loss_dash_dash = -1
        
        
        return loss,loss_dash,loss_dash_dash
    

In [6]:
class Neural_Layer:
    '''
    This is a very specefic Neuron class that uses sigmoid activation
    and square loss.
    '''
    def __init__(self,l0,l1):
        self.W = np.random.randn(l0 ,l1).astype(np.float128) * np.sqrt(2.0 /l0 )
        self.b = np.zeros((l1)).astype(np.float128)
        print(self.W.shape , self.b.shape)
        
    def forward(self,X):
        #print(X.shape , self.W.shape , self.b.shape)
        h_x = X.dot(self.W) + self.b
        h_x = sigmoid(h_x)
        return h_x
    
    def update_newton(self,X,h_x,d_back , d_back_dash):
        #f_x = self.forward(X)
        f_x = h_x
        
        #Derivating w.r.t W
        temp = f_x * (1 - f_x) #this value was computed again and again
        f_x_dash_W = X.T.dot(temp * d_back)
        
        first_term = X.T.dot(h_x * (1-h_x) * d_back_dash) * X.T.dot(h_x * (1-h_x))
        second_term = (X*X).T.dot(d_back * (h_x * (1 - h_x) * (1 - 2 * h_x)))
        f_x_dash_dash_W =  first_term + second_term
        
        #Derivating w.r.t b
        f_x_dash_b = temp * d_back
        
        first_term = d_back_dash * np.square(temp)
        second_term = d_back * temp * (1 - 2 * h_x)
        f_x_dash_dash_b = first_term + second_term
        
        #Derivating w.r.t x(to return to previous layers)
        f_x_dash_x = (temp * d_back).dot(self.W.T)
        
        first_term = (d_back_dash * temp).dot(self.W.T) * temp.dot(self.W.T)
        second_term = (d_back * h_x * (1 - h_x ) * (1 - 2 * h_x)).dot(np.square(self.W).T)
        f_x_dash_dash_x = first_term + second_term
        
        delta_W = -1 * (f_x_dash_W / f_x_dash_dash_W)
        self.W = self.W + delta_W
        
        delta_b = -1 * np.sum(f_x_dash_b / f_x_dash_dash_b , axis = 0)
        self.b = self.b + delta_b
        
        d_back = f_x_dash_x
        d_back_dash = f_x_dash_dash_x
        
        return d_back,d_back_dash
        
    def update_batch_gradient_descent(self , X , h_x , d_back,alpha = 0.01):
        
        f_x = h_x
        
        temp = f_x * (1 - f_x) * d_back
        #Derivating w.r.t W
        f_x_dash_W = X.T.dot(temp)
        
        #Derivatng w.r.t b
        f_x_dash_b = temp
        
        #Derivating w.r.t x(to return to previous layers)
        f_x_dash_x = temp.dot(self.W.T)
        
        #Update W
        delta_W = f_x_dash_W
        self.W = self.W - alpha * delta_W
        
        #Update b
        delta_b = np.sum(f_x_dash_b , axis =0)
        self.b = self.b - alpha * delta_b
        
        d_back = f_x_dash_x
        return d_back
           

In [7]:
def accuracy(model , X , Y):
    pred = model.forward(X)
    pred =pred > 0.5
    acc = np.sum(pred == Y)
    acc = acc / Y.shape[0]
    return acc

In [8]:
# Declare a neuron with shape of weights as [shape_of_input,1]
model = Neural_Layer(2,1)
loss_m = Loss()

(2, 1) (1,)


In [9]:
# testing the model outputs
model.forward(X_train)

array([[0.5       ],
       [0.41036746],
       [0.54326179],
       [0.45289868]], dtype=float128)

In [10]:
# Now we can train the model by training on a batch.As our batch is equal to the raing data.1 iter = 1 epoch


def train_newton_update(model ,loss_m,X_train , Y_train,epochs):
    for i in range(epochs):
        h_x = model.forward(X_train)
        loss,d_loss,d_loss_dash = loss_m.sq_loss(h_x,Y_train)
        acc = accuracy(model ,X_train,Y_train)
        print("The training loss at {}th epoch : {}  Training Accuracy:{}".format(i , loss , acc))
        
        model.update_newton(X_train,h_x , d_loss,d_loss_dash)        

train_newton_update(model , loss_m , X_train,Y_train , 2)
    

The training loss at 0th epoch : 0.5064273411256793  Training Accuracy:0.25
The training loss at 1th epoch : 0.516434484138249  Training Accuracy:0.75


In [11]:
# Now we can predict the values for unseen data or trained data also
# We can also calculate the accuracy of the model we have trained
accuracy(model , X_train,Y_train)

0.5

In [12]:
model = Neural_Layer(2,1)
loss_m = Loss()

def train_gradient_update(model ,loss_m,X_train , Y_train,epochs):
    for i in range(epochs):
        h_x = model.forward(X_train)
        loss,d_loss,_ = loss_m.sq_loss(h_x,Y_train)
        acc = accuracy(model ,X_train,Y_train)
        print("The training loss at {}th epoch : {}  Training Accuracy:{}".format(i , loss , acc))
        
        model.update_batch_gradient_descent(X_train,h_x , d_loss , 0.01)        

train_gradient_update(model , loss_m , X_train,Y_train , 10)
    

(2, 1) (1,)
The training loss at 0th epoch : 0.5035487222282764  Training Accuracy:0.5
The training loss at 1th epoch : 0.5035441470201979  Training Accuracy:0.5
The training loss at 2th epoch : 0.5035395796637846  Training Accuracy:0.5
The training loss at 3th epoch : 0.503535020130299  Training Accuracy:0.5
The training loss at 4th epoch : 0.5035304683911919  Training Accuracy:0.5
The training loss at 5th epoch : 0.5035259244180997  Training Accuracy:0.5
The training loss at 6th epoch : 0.5035213881828448  Training Accuracy:0.5
The training loss at 7th epoch : 0.5035168596574321  Training Accuracy:0.5
The training loss at 8th epoch : 0.5035123388140494  Training Accuracy:0.5
The training loss at 9th epoch : 0.5035078256250654  Training Accuracy:0.5


In [13]:
# Now we can predict the values for unseen data or trained data also
# We can also calculate the accuracy of the model we have trained
accuracy(model , X_train,Y_train)

0.5

# Neural Network

Now that we have built a single perceptron layer we can scale it to make a neural net(A deep Neral Net would just be adding more layers on it)

We will try to make a Neural Network that is scalable from scratch.That is it can be directly used as a library.

In [14]:
class Neural_Net:
    
    def __init__(self , layer_list):
        
        
        self.hidden_layers = len(layer_list) - 1
        
        self.layers = {}
        
        for i in range(self.hidden_layers):            
            self.layers[i+1] = Neural_Layer(layer_list[i] , layer_list[i+1])
            
            
    def forward(self , X_train):
        t = X_train
        cache = []
        cache.append(t)        
        for i in range(self.hidden_layers):
            t = self.layers[i+1].forward(t)
            cache.append(t)
            
        return t,cache
    
    def update_newton(self , cache, d_back,d_back_dash):
        #list of activations--cache
        
        for i in range(self.hidden_layers , 0 , -1):
            d_back,d_back_dash = self.layers[i].update_newton(cache[i-1],cache[i],d_back,d_back_dash)
            
        return d_back,d_back_dash 
    
    def update_gradient(self , cache, d_back , alpha=0.01):
        for i in range(self.hidden_layers , 0 , -1):
            d_back = self.layers[i].update_batch_gradient_descent(cache[i-1],cache[i],d_back,alpha)
            
        return d_back 
            

In [15]:
def accuracy_neural_net(model , X , Y):
    pred,_ = model.forward(X)
    pred =pred > 0.5
    acc = np.sum(pred == Y)
    acc = acc / Y.shape[0]
    return acc

In [16]:
# Lets make a neural net as follows
# visible_layer-->3 neurons-->1 neuron
layer_list = [2 , 3, 1]
model = Neural_Net(layer_list)
loss_m = Loss()

(2, 3) (3,)
(3, 1) (1,)


In [17]:
pred,_ = model.forward(X_train)
pred

array([[0.28102243],
       [0.38233196],
       [0.27124511],
       [0.35378332]], dtype=float128)

In [18]:
def train_newton_update(model ,loss_m,X_train , Y_train,epochs):
    for i in range(epochs):
        act , cache = model.forward(X_train)
        loss,d_back,d_back_dash = loss_m.sq_loss(act,Y_train)
        acc = accuracy_neural_net(model ,X_train,Y_train)
        print("The training loss at {}th epoch : {}  Training Accuracy:{}".format(i , loss , acc))
        model.update_newton(cache,d_back,d_back_dash)        

train_newton_update(model , loss_m , X_train,Y_train , 2)
    

The training loss at 0th epoch : 0.5771381906037006  Training Accuracy:0.5
The training loss at 1th epoch : 0.632520062156091  Training Accuracy:0.5


In [19]:
accuracy_neural_net(model,X_train, Y_train)

0.5

In [23]:
layer_list = [2 ,3, 1]
model = Neural_Net(layer_list)
loss_m = Loss()

def train_gradient_update(model ,loss_m,X_train , Y_train,epochs,alpha =0.1):
    for i in range(epochs):
        act , cache = model.forward(X_train)
        loss,d_back,_ = loss_m.sq_loss(act,Y_train)
        acc = accuracy_neural_net(model ,X_train,Y_train)
        print("The training loss at {}th epoch : {}  Training Accuracy:{}".format(i , loss , acc))
        model.update_gradient(cache,d_back,alpha)        

train_gradient_update(model , loss_m , X_train,Y_train , 2000)

(2, 3) (3,)
(3, 1) (1,)
The training loss at 0th epoch : 0.8528249546211842  Training Accuracy:0.5
The training loss at 1th epoch : 0.8492732557401613  Training Accuracy:0.5
The training loss at 2th epoch : 0.8455912896380257  Training Accuracy:0.5
The training loss at 3th epoch : 0.841773865269743  Training Accuracy:0.5
The training loss at 4th epoch : 0.8378156820199799  Training Accuracy:0.5
The training loss at 5th epoch : 0.8337113496857546  Training Accuracy:0.5
The training loss at 6th epoch : 0.8294554133471445  Training Accuracy:0.5
The training loss at 7th epoch : 0.8250423838438584  Training Accuracy:0.5
The training loss at 8th epoch : 0.8204667746378649  Training Accuracy:0.5
The training loss at 9th epoch : 0.8157231458989778  Training Accuracy:0.5
The training loss at 10th epoch : 0.810806156696766  Training Accuracy:0.5
The training loss at 11th epoch : 0.8057106262125071  Training Accuracy:0.5
The training loss at 12th epoch : 0.800431604891752  Training Accuracy:0.5
T

The training loss at 707th epoch : 0.4133761143352003  Training Accuracy:0.75
The training loss at 708th epoch : 0.4132480568456696  Training Accuracy:0.75
The training loss at 709th epoch : 0.4131199953434475  Training Accuracy:0.75
The training loss at 710th epoch : 0.4129919301746582  Training Accuracy:0.75
The training loss at 711th epoch : 0.4128638616848706  Training Accuracy:0.75
The training loss at 712th epoch : 0.41273579021908946  Training Accuracy:0.75
The training loss at 713th epoch : 0.4126077161217465  Training Accuracy:0.75
The training loss at 714th epoch : 0.41247963973669155  Training Accuracy:0.75
The training loss at 715th epoch : 0.41235156140718365  Training Accuracy:0.75
The training loss at 716th epoch : 0.4122234814758825  Training Accuracy:0.75
The training loss at 717th epoch : 0.4120954002848394  Training Accuracy:0.75
The training loss at 718th epoch : 0.41196731817548876  Training Accuracy:0.75
The training loss at 719th epoch : 0.41183923548863954  Trai

The training loss at 1367th epoch : 0.33606678016847763  Training Accuracy:0.75
The training loss at 1368th epoch : 0.3359593556345831  Training Accuracy:0.75
The training loss at 1369th epoch : 0.33585192015898563  Training Accuracy:0.75
The training loss at 1370th epoch : 0.335744473406234  Training Accuracy:0.75
The training loss at 1371th epoch : 0.3356370150398493  Training Accuracy:0.75
The training loss at 1372th epoch : 0.335529544722322  Training Accuracy:0.75
The training loss at 1373th epoch : 0.3354220621151099  Training Accuracy:0.75
The training loss at 1374th epoch : 0.33531456687863526  Training Accuracy:0.75
The training loss at 1375th epoch : 0.3352070586722825  Training Accuracy:0.75
The training loss at 1376th epoch : 0.3350995371543957  Training Accuracy:0.75
The training loss at 1377th epoch : 0.3349920019822761  Training Accuracy:0.75
The training loss at 1378th epoch : 0.3348844528121792  Training Accuracy:0.75
The training loss at 1379th epoch : 0.3347768892993

In [24]:
pred,_ = model.forward(X_train)
pred

array([[0.78167059],
       [0.36434847],
       [0.29372793],
       [0.52631773]], dtype=float128)

In [25]:
accuracy_neural_net(model,X_train, Y_train)

1.0

# N Bit XOR

Now Lets try working with just a little better data. A n XOR operator. So lets create the dataset for n bit xor.

We would have atmost 2^n data point in this type of data set.BUt we would limit our dataset to a 1000 data points
whichever is smaller.

Then we can divide into training and test set

In [26]:

n = 5
max_datapoint = 10000
datapoints = min(pow(2,n) , max_datapoint)

X = np.zeros((datapoints , n) , dtype=np.int32)
Y = np.zeros((datapoints , 1), dtype=np.int32)

for i in range(datapoints):
    tmp = i
    y_tmp = 0
    for j in range(n-1 , -1 , -1):
        X[i,j] = tmp&1
        y_tmp = y_tmp^X[i,j]
        tmp = tmp>>1
    Y[i] = y_tmp
        
        

In [27]:
# for sanity check lets print one example
ind = 11
print(X[ind] , Y[ind])

[0 1 0 1 1] [1]


In [28]:
# Lets divide the set in training and testing
div = 0.9
train_n = int(div * datapoints)
X_train = X[:train_n]
Y_train = Y[:train_n]

X_test = X[train_n:]
Y_test = Y[train_n:]
Y_test.shape

(4, 1)

In [29]:
layer_list = [n,4,2,1]
model = Neural_Net(layer_list)
loss_m = Loss()

(5, 4) (4,)
(4, 2) (2,)
(2, 1) (1,)


In [30]:
train_gradient_update(model , loss_m , X_train,Y_train , 20000)

The training loss at 0th epoch : 4.2088408504880235  Training Accuracy:0.5
The training loss at 1th epoch : 3.9665384877213765  Training Accuracy:0.5
The training loss at 2th epoch : 3.789999719313251  Training Accuracy:0.5
The training loss at 3th epoch : 3.673684337637267  Training Accuracy:0.5
The training loss at 4th epoch : 3.6022548568274537  Training Accuracy:0.5
The training loss at 5th epoch : 3.5602133030678544  Training Accuracy:0.5
The training loss at 6th epoch : 3.5360190580230437  Training Accuracy:0.5
The training loss at 7th epoch : 3.5222385813969197  Training Accuracy:0.5
The training loss at 8th epoch : 3.5144178217325446  Training Accuracy:0.5
The training loss at 9th epoch : 3.5099798771322197  Training Accuracy:0.5
The training loss at 10th epoch : 3.507457210899727  Training Accuracy:0.5
The training loss at 11th epoch : 3.506019100296306  Training Accuracy:0.5
The training loss at 12th epoch : 3.5051959212630743  Training Accuracy:0.5
The training loss at 13th 

The training loss at 518th epoch : 3.501196509413828  Training Accuracy:0.5
The training loss at 519th epoch : 3.50119376119547  Training Accuracy:0.5
The training loss at 520th epoch : 3.5011910188316127  Training Accuracy:0.5
The training loss at 521th epoch : 3.501188282303707  Training Accuracy:0.5
The training loss at 522th epoch : 3.5011855515932764  Training Accuracy:0.5
The training loss at 523th epoch : 3.5011828266819145  Training Accuracy:0.5
The training loss at 524th epoch : 3.5011801075512867  Training Accuracy:0.5
The training loss at 525th epoch : 3.5011773941831286  Training Accuracy:0.5
The training loss at 526th epoch : 3.5011746865592457  Training Accuracy:0.5
The training loss at 527th epoch : 3.501171984661514  Training Accuracy:0.5
The training loss at 528th epoch : 3.501169288471879  Training Accuracy:0.5
The training loss at 529th epoch : 3.5011665979723543  Training Accuracy:0.5
The training loss at 530th epoch : 3.501163913145024  Training Accuracy:0.5
The tr

The training loss at 1059th epoch : 3.5002454030117476  Training Accuracy:0.5357142857142857
The training loss at 1060th epoch : 3.500244250067149  Training Accuracy:0.5357142857142857
The training loss at 1061th epoch : 3.500243098418802  Training Accuracy:0.5357142857142857
The training loss at 1062th epoch : 3.5002419480633526  Training Accuracy:0.5357142857142857
The training loss at 1063th epoch : 3.500240798997455  Training Accuracy:0.5357142857142857
The training loss at 1064th epoch : 3.5002396512177723  Training Accuracy:0.5357142857142857
The training loss at 1065th epoch : 3.5002385047209756  Training Accuracy:0.5357142857142857
The training loss at 1066th epoch : 3.500237359503745  Training Accuracy:0.5357142857142857
The training loss at 1067th epoch : 3.500236215562769  Training Accuracy:0.5357142857142857
The training loss at 1068th epoch : 3.5002350728947453  Training Accuracy:0.5357142857142857
The training loss at 1069th epoch : 3.5002339314963793  Training Accuracy:0

The training loss at 1541th epoch : 3.499792335360264  Training Accuracy:0.5714285714285714
The training loss at 1542th epoch : 3.4997915327905527  Training Accuracy:0.5714285714285714
The training loss at 1543th epoch : 3.499790730544359  Training Accuracy:0.5714285714285714
The training loss at 1544th epoch : 3.499789928620496  Training Accuracy:0.5714285714285714
The training loss at 1545th epoch : 3.499789127017779  Training Accuracy:0.5714285714285714
The training loss at 1546th epoch : 3.499788325735025  Training Accuracy:0.5714285714285714
The training loss at 1547th epoch : 3.4997875247710537  Training Accuracy:0.5714285714285714
The training loss at 1548th epoch : 3.4997867241246867  Training Accuracy:0.5714285714285714
The training loss at 1549th epoch : 3.499785923794747  Training Accuracy:0.5714285714285714
The training loss at 1550th epoch : 3.4997851237800606  Training Accuracy:0.5714285714285714
The training loss at 1551th epoch : 3.4997843240794544  Training Accuracy:0.

The training loss at 2114th epoch : 3.499355141358984  Training Accuracy:0.6071428571428571
The training loss at 2115th epoch : 3.499354373292097  Training Accuracy:0.6071428571428571
The training loss at 2116th epoch : 3.4993536050717604  Training Accuracy:0.6071428571428571
The training loss at 2117th epoch : 3.4993528366973368  Training Accuracy:0.6071428571428571
The training loss at 2118th epoch : 3.4993520681681884  Training Accuracy:0.6428571428571429
The training loss at 2119th epoch : 3.4993512994836795  Training Accuracy:0.6428571428571429
The training loss at 2120th epoch : 3.4993505306431736  Training Accuracy:0.6428571428571429
The training loss at 2121th epoch : 3.4993497616460347  Training Accuracy:0.6428571428571429
The training loss at 2122th epoch : 3.4993489924916266  Training Accuracy:0.6428571428571429
The training loss at 2123th epoch : 3.4993482231793136  Training Accuracy:0.6428571428571429
The training loss at 2124th epoch : 3.4993474537084612  Training Accurac

...

The training loss at 17839th epoch : 0.41525028974786476  Training Accuracy:0.9642857142857143
The training loss at 17840th epoch : 0.41524340722138636  Training Accuracy:0.9642857142857143
The training loss at 17841th epoch : 0.41523653018408613  Training Accuracy:0.9642857142857143
The training loss at 17842th epoch : 0.4152296586297624  Training Accuracy:0.9642857142857143
The training loss at 17843th epoch : 0.41522279255222244  Training Accuracy:0.9642857142857143
The training loss at 17844th epoch : 0.41521593194528245  Training Accuracy:0.9642857142857143
The training loss at 17845th epoch : 0.4152090768027676  Training Accuracy:0.9642857142857143
The training loss at 17846th epoch : 0.415202227118512  Training Accuracy:0.9642857142857143
The training loss at 17847th epoch : 0.41519538288635865  Training Accuracy:0.9642857142857143
The training loss at 17848th epoch : 0.41518854410015943  Training Accuracy:0.9642857142857143
The training loss at 17849th epoch : 0.415181710753775

The training loss at 18359th epoch : 0.41229044162136763  Training Accuracy:0.9642857142857143
The training loss at 18360th epoch : 0.4122857465399385  Training Accuracy:0.9642857142857143
The training loss at 18361th epoch : 0.41228105464456993  Training Accuracy:0.9642857142857143
The training loss at 18362th epoch : 0.4122763659321751  Training Accuracy:0.9642857142857143
The training loss at 18363th epoch : 0.4122716803996708  Training Accuracy:0.9642857142857143
The training loss at 18364th epoch : 0.41226699804397804  Training Accuracy:0.9642857142857143
The training loss at 18365th epoch : 0.41226231886202136  Training Accuracy:0.9642857142857143
The training loss at 18366th epoch : 0.4122576428507292  Training Accuracy:0.9642857142857143
The training loss at 18367th epoch : 0.41225297000703387  Training Accuracy:0.9642857142857143
The training loss at 18368th epoch : 0.41224830032787146  Training Accuracy:0.9642857142857143
The training loss at 18369th epoch : 0.412243633810181

The training loss at 18894th epoch : 0.410166511679655  Training Accuracy:0.9642857142857143
The training loss at 18895th epoch : 0.4101631612076154  Training Accuracy:0.9642857142857143
The training loss at 18896th epoch : 0.4101598126999873  Training Accuracy:0.9642857142857143
The training loss at 18897th epoch : 0.4101564661551139  Training Accuracy:0.9642857142857143
The training loss at 18898th epoch : 0.41015312157134015  Training Accuracy:0.9642857142857143
The training loss at 18899th epoch : 0.4101497789470128  Training Accuracy:0.9642857142857143
The training loss at 18900th epoch : 0.4101464382804804  Training Accuracy:0.9642857142857143
The training loss at 18901th epoch : 0.41014309957009326  Training Accuracy:0.9642857142857143
The training loss at 18902th epoch : 0.4101397628142036  Training Accuracy:0.9642857142857143
The training loss at 18903th epoch : 0.4101364280111653  Training Accuracy:0.9642857142857143
The training loss at 18904th epoch : 0.410133095159334  Tra

The training loss at 19272th epoch : 0.4090265782927107  Training Accuracy:0.9642857142857143
The training loss at 19273th epoch : 0.4090238666025581  Training Accuracy:0.9642857142857143
The training loss at 19274th epoch : 0.4090211563602438  Training Accuracy:0.9642857142857143
The training loss at 19275th epoch : 0.40901844756464983  Training Accuracy:0.9642857142857143
The training loss at 19276th epoch : 0.40901574021465936  Training Accuracy:0.9642857142857143
The training loss at 19277th epoch : 0.40901303430915675  Training Accuracy:0.9642857142857143
The training loss at 19278th epoch : 0.4090103298470273  Training Accuracy:0.9642857142857143
The training loss at 19279th epoch : 0.40900762682715747  Training Accuracy:0.9642857142857143
The training loss at 19280th epoch : 0.4090049252484349  Training Accuracy:0.9642857142857143
The training loss at 19281th epoch : 0.4090022251097483  Training Accuracy:0.9642857142857143
The training loss at 19282th epoch : 0.4089995264099875 

The training loss at 19758th epoch : 0.40786036280669075  Training Accuracy:0.9642857142857143
The training loss at 19759th epoch : 0.40785824145052135  Training Accuracy:0.9642857142857143
The training loss at 19760th epoch : 0.4078561211081771  Training Accuracy:0.9642857142857143
The training loss at 19761th epoch : 0.40785400177895215  Training Accuracy:0.9642857142857143
The training loss at 19762th epoch : 0.4078518834621413  Training Accuracy:0.9642857142857143
The training loss at 19763th epoch : 0.40784976615704016  Training Accuracy:0.9642857142857143
The training loss at 19764th epoch : 0.40784764986294475  Training Accuracy:0.9642857142857143
The training loss at 19765th epoch : 0.40784553457915185  Training Accuracy:0.9642857142857143
The training loss at 19766th epoch : 0.4078434203049588  Training Accuracy:0.9642857142857143
The training loss at 19767th epoch : 0.4078413070396637  Training Accuracy:0.9642857142857143
The training loss at 19768th epoch : 0.407839194782565

In [31]:
print("Training accuracy:{}".format(accuracy_neural_net(model,X_train, Y_train)))

print("Test accuracy:{}".format(accuracy_neural_net(model,X_test, Y_test)))


Training accuracy:0.9642857142857143
Test accuracy:1.0


In [32]:
layer_list = [n,4,2,1]
model = Neural_Net(layer_list)
loss_m = Loss()
train_newton_update(model , loss_m , X_train,Y_train , 2)

(5, 4) (4,)
(4, 2) (2,)
(2, 1) (1,)
The training loss at 0th epoch : 3.607285466475095  Training Accuracy:0.5
The training loss at 1th epoch : 6.998835234621003  Training Accuracy:0.5


In [33]:
accuracy_neural_net(model,X_test, Y_test)

0.5